## Problem: Library Book Circulation Tracker

Date: 8/21/2025

**Your task is to develop a library book circulation tracker.**

You are given a sequence of `operations` that represent activities in a library. Each operation is one of three types: **acquisition**, **checkout**, or **reclassify**. Operations are provided in the following format:

### Operation Types:

1. **["acquisition", "<book_category>", "<quantity>", "<price>"]** 
   - The library acquires `<quantity>` books of `<book_category>`, each valued at `<price>` for insurance purposes.

2. **["checkout", "<book_category>", "<quantity>"]** 
   - Patrons borrow `<quantity>` books of `<book_category>`. 
   - If books of the specified category have different insurance values, **the least valuable ones should be checked out first**. 
   - It is guaranteed that the library will always have enough books to fulfill all checkout requests.

3. **["reclassify", "<book_category>", "<quantity>", "<original_price>", "<new_price>"]** 
   - The library reclassifies `<quantity>` books of `<book_category>` to a more valuable edition. 
   - It is guaranteed that there are `<quantity>` books of the specified category with the `<original_price>` value.

**Your function should calculate the total insurance value of all books checked out after processing the entire sequence of operations.**

**Return an array representing the insurance value of books for each checkout operation.**

**Note:** You are not expected to provide the most optimal solution, but a solution with time complexity not worse than O(operations.length²) will fit within the execution time limit.

---

## Example

**For:**
```python
operations = [
    ["acquisition", "fiction", "2", "100"],
    ["acquisition", "reference", "3", "60"],
    ["checkout", "fiction", "1"],
    ["checkout", "reference", "1"],
    ["reclassify", "reference", "1", "60"],
    ["checkout", "reference", "1"],
    ["checkout", "reference", "1"]
]
```

the output should be `solution(operations) = [100, 60, 60, 100]`

**Let's analyze the operations:**

1. **["acquisition", "fiction", "2", "100"]** - the library acquires 2 fiction books, each valued at 100.

2. **["acquisition", "reference", "3", "60"]** - the library acquires 3 reference books, each valued at 60.

3. **["checkout", "fiction", "1"]** - a patron checks out 1 fiction book valued at 100, so the insurance value is 1 × 100 = 100.

4. **["checkout", "reference", "1"]** - a patron checks out 1 reference book valued at 60, so the insurance value is 60.

5. **["reclassify", "reference", "1", "60", ...** (appears cut off but likely increases value)

6. **["checkout", "reference", "1"]** - checkout continues...

---

## Analysis

This is a **priority queue / heap problem**! 

**Key insights:**
1. Need to track books by category and price
2. Checkout always takes **least valuable** books first (min-heap!)
3. Reclassify modifies existing book values

**Data structure:**
- Dictionary mapping `category` → min-heap of prices
- Each heap element: price (with count if tracking quantities)

In [ ]:
from heapq import heappush, heappop

def solution(operations):
    categories = {}
    result = []
    
    for op in operations:
        op_type, category = op[0], op[1]
        
        if category not in categories:
            categories[category] = [[], {}]  # [heap, counts]
        
        heap, counts = categories[category]
        
        if op_type == "acquisition":
            qty, price = int(op[2]), int(op[3])
            counts[price] = counts.get(price, 0) + qty
            heappush(heap, price)
        
        elif op_type == "reclassify":
            qty, old_p, new_p = int(op[2]), int(op[3]), int(op[4])
            counts[old_p] -= qty
            counts[new_p] = counts.get(new_p, 0) + qty
            heappush(heap, new_p)
        
        else:  # checkout
            qty = int(op[2])
            total = 0
            
            while qty > 0:
                # Clean expired prices
                while heap and counts.get(heap[0], 0) == 0:
                    heappop(heap)
                
                price = heap[0]
                take = min(qty, counts[price])
                total += take * price
                counts[price] -= take
                qty -= take
                
                if counts[price] == 0:
                    heappop(heap)
            
            result.append(total)
    
    return result